In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import os
import sys

parent_directory = os.path.abspath('..')
sys.path.append(parent_directory)

from actionweaver.llms import patch
from langchain_community.utilities.github import GitHubAPIWrapper
from langsmith.run_helpers import traceable

from autocoder.bot import AutoCoder
from autocoder.index import RepositoryIndex

from autocoder.codebase import Codebase

In [7]:
github_repository = "TengHu/AutoCoder"
github_api = GitHubAPIWrapper(
    github_repository=github_repository,
    github_app_id=os.environ["GITHUB_APP_ID"],
    github_app_private_key=os.environ["GITHUB_APP_PRIVATE_KEY"],
)
codebase = Codebase(github_api)
index = RepositoryIndex(github_api, github_repository, codebase)

Indexing codebase TengHu/AutoCoder
Indexing autocoder/__init__.py
Indexing autocoder/bot.py
Indexing autocoder/codebase.py
Indexing autocoder/pydantic_models.py
Indexing autocoder/pydantic_models_v2.py
Indexing autocoder/rag.py
Indexing autocoder/telemetry.py
Indexing main.py


In [8]:
auto_coder = AutoCoder(github_api, index, codebase)

[System] Branch 'aw_demo_bot_v4' created successfully, and set as current active branch.


In [9]:
res = auto_coder("Create two_sum.py with 3 solutions for two sum problem")
print(res)

The `two_sum.py` file has been created with three different solutions for the two-sum problem. Below is the content of the file:

```python
# Solution 1: Brute Force Approach
# Time Complexity: O(n^2)
def two_sum_brute_force(nums, target):
    for i in range(len(nums)):
        for j in range(i + 1, len(nums)):
            if nums[i] + nums[j] == target:
                return [i, j]
    return []

# Solution 2: Hash Table Approach
# Time Complexity: O(n)
def two_sum_hash_table(nums, target):
    hash_table = {}
    for i, num in enumerate(nums):
        complement = target - num
        if complement in hash_table:
            return [hash_table[complement], i]
        hash_table[num] = i
    return []

# Solution 3: Two-Pointer Technique for Sorted Array
# Time Complexity: O(n)
# Note: This solution assumes the input array is already sorted.
def two_sum_two_pointer(nums, target):
    left, right = 0, len(nums) - 1
    while left < right:
        current_sum = nums[left] + nums[right]

In [ ]:
auto_coder("create a pr")

In [11]:
nodes = index.query("AutoCoder")

In [29]:
from openai.types.chat.chat_completion_message import ChatCompletionMessage
from openai.types.chat.chat_completion_message_tool_call import (
    ChatCompletionMessageToolCall,
Function
)
messages = [{'role': 'system',
  'content': 'You are a coding assistant, you have the capability to assist with code-related tasks and modify files.'},
 {'role': 'user', 'content': 'Move trace_client function to bot.py.'},
 ChatCompletionMessage(content=None, role='assistant', tool_calls=[ChatCompletionMessageToolCall(id='call_9f2agXdE5nnUl1X04XkYHjeQ', function=Function(arguments='{"description":"Move the \'trace_client\' function from its current location to the \'bot.py\' file."}', name='PlanCodeChange'), type='function')]),
 {'tool_call_id': 'call_9f2agXdE5nnUl1X04XkYHjeQ',
  'role': 'tool',
  'name': 'PlanCodeChange',
  'content': "- New files created: []\n        - Existing files updated: [['Updated file autocoder/bot.py', 'Updated file autocoder/telemetry.py']]\n        - Problems encountered: []\n        ###\n    Ignore others,    Make the previous message more user-friendly\n        "}
           ]

In [30]:
from openai import OpenAI


client = OpenAI()
response = client.chat.completions.create(
            model="gpt-4-1106-preview",
            messages=messages,
            stream=False,
        )

In [31]:
print (response)

ChatCompletion(id='chatcmpl-8h2sHwJwIR8SOkWxw2ypBGd80Uyom', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='I can assist you with moving the `trace_client` function to `bot.py`, but I need access to the source code or need to know where the current `trace_client` function is located. Can you provide the current code file that contains the `trace_client` function or tell me where to find it?', role='assistant', function_call=None, tool_calls=None))], created=1705269845, model='gpt-4-1106-preview', object='chat.completion', system_fingerprint='fp_168383a679', usage=CompletionUsage(completion_tokens=63, prompt_tokens=135, total_tokens=198))


In [32]:
print (response.choices[0].message.content)

I can assist you with moving the `trace_client` function to `bot.py`, but I need access to the source code or need to know where the current `trace_client` function is located. Can you provide the current code file that contains the `trace_client` function or tell me where to find it?


In [17]:
print (messages[0]['content'])

You are a coding assistant, you have the capability to assist with code-related tasks and modify files.


In [18]:
messages

[{'role': 'system',
  'content': 'You are a coding assistant, you have the capability to assist with code-related tasks and modify files.'},
 {'role': 'user', 'content': 'Move trace_client function to bot.py.'},
 ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_9f2agXdE5nnUl1X04XkYHjeQ', function=Function(arguments='{"description":"Move the \'trace_client\' function from its current location to the \'bot.py\' file."}', name='PlanCodeChange'), type='function')]),
 {'tool_call_id': 'call_9f2agXdE5nnUl1X04XkYHjeQ',
  'role': 'tool',
  'name': 'PlanCodeChange',
  'content': "I've read code and attempted to create code changes. The outcome of these code changes is as follows:\n        - New files created: []\n        - Existing files updated: [['Updated file autocoder/bot.py', 'Updated file autocoder/telemetry.py']]\n        - Problems encountered: []\n        ###\n        Make the previous message more user-friendl

In [8]:
user_prompt = '''
CONTEXT FOR MAKING CODE MODIFICATIONS:

###############
This section is divider and not a part of the code.
Code Snippet From Filepath: autocoder/bot.py
#############

class AutoCoder:
<END OF SNIPPET>
###############
This section is divider and not a part of the code.
Code Snippet From Filepath: autocoder/bot.py
#############

import datetime
import os
import uuid
from typing import List, Union

from actionweaver import action
from actionweaver.utils.tokens import TokenUsageTracker
from langchain_community.utilities.github import GitHubAPIWrapper
from langsmith.run_helpers import traceable
from llama_index import Document, ServiceContext, VectorStoreIndex
from llama_index.node_parser import CodeSplitter
from openai import AzureOpenAI, OpenAI
from pydantic import BaseModel, Field

from autocoder.pydantic_models import create_context, create_tasks
from autocoder.telemetry import trace_client

assert os.environ["MODEL"]
MODEL = os.environ["MODEL"]


DIVIDING_LINE = """
###############
This section is divider and not a part of the code.
{input}
#############

"""
<END OF SNIPPET>
###############
This section is divider and not a part of the code.
Code Snippet From Filepath: autocoder/bot.py
#############

@traceable(run_type="tool")
    def gather_context(self, input):
        user_prompt = input

        messages = [
            # {
            #     "role": "system",
            #     "content": "You are good at extract information from description",
            # },
            {"role": "user", "content": f"Description: {user_prompt}"},
        ]
        context = create_context.invoke(
            self.client,
            messages=messages,
            model=MODEL,
            stream=False,
            force=True,
        )

        if isinstance(context, list):
            context = context[0]

        index_response = ""
        for query in context.queries + context.instructions:
            nodes = self.index.query(query)
            for node in nodes:
                index_response = (
                    index_response
                    + DIVIDING_LINE.format(
                        input=f"Code Snippet From Filepath: {node.metadata['file']}"
                    )
                    + f"{node.text}\n"
                    + "<END OF SNIPPET>"
                )

        file_response = self.read_files(context.files)
        code_search_response = self.search_code(" ".join(context.code_snippets))

        return (
            "CONTEXT FOR MAKING CODE MODIFICATIONS:\n"
            + index_response
            + "\n"
            + file_response
            + "\n"
            + code_search_response
        )
<END OF SNIPPET>
###############
This section is divider and not a part of the code.
Code Snippet From Filepath: autocoder/bot.py
#############

@action(name="QuestionAnswer", decorators=[traceable(run_type="tool")])
    def question_answer(self, rewritten_query: str, keywords: List[str]):
        """Answer questions about the codebase"""

        context = self.gather_context(" ".join(keywords))

        messages = [
            {
                "role": "user",
                "content": f"{context} \n###########\n Question: {rewritten_query}",
            }
        ]
        response = self.client.chat.completions.create(
            model=MODEL,
            messages=messages,
            stream=False,
            token_usage_tracker=TokenUsageTracker(500),
        )
        return response

    @action(name="GetIssues", decorators=[traceable(run_type="tool")])
    def get_issues(self):
        """
        Get a list of issues from the GitHub repo.
        """
        response = self.github_api.get_issues()
        response = response.split("\n")
        return eval(response[1]) if len(response) > 1 else []

    @action(name="CreateGitBranch", decorators=[traceable(run_type="tool")])
    def create_branch(self, branch: str):
        """
        Create a new Git branch.
        """
        return self.github_api.create_branch(branch)
<END OF SNIPPET>
###############
This section is divider and not a part of the code.
Code Snippet From Filepath: autocoder/bot.py
#############

@action("PlanCodeChange", stop=True, decorators=[traceable(run_type="tool")])
    def plan_code_change(self, description: str):
        """
        Plan code changes based on a given description.

        This method is designed to handle various types of code alterations such as
        inserting new code, refactoring existing code, replacing segments, or making
        general modifications.
        """
        context = self.gather_context(description)

        user_prompt = f"""
        {context}
        {'#' * 20}
        Description:
        {description}"""

        messages = [{"role": "user", "content": user_prompt}]
        tasks = create_tasks.invoke(
            self.client,
            messages=messages,
            temperature=0.1,
            model=MODEL,
            stream=False,
            force=True,
        )

        if isinstance(tasks, list):
            tasks = tasks[0]
        messages = tasks.execute(self.client, self.github_api, context)

        files_updated = []
        files_created = []
        problems = []
        for msg in messages:
            if "Updated file" in str(msg):
                files_updated.append(msg)
            elif "Created file" in str(msg):
                files_created.append(msg)
            else:
                problems.append(msg)

        return self.rephrase(
            f"""
- New files created: {files_created}
- Existing files updated: {files_updated}
- Problems encountered: {problems}
"""
        )
<END OF SNIPPET>
###############
This section is divider and not a part of the code.
Code Snippet From Filepath: autocoder/bot.py
#############

def search_code(self, query: str):
        return self.github_api.search_code(query)
<END OF SNIPPET>
###############
This section is divider and not a part of the code.
Code Snippet From Filepath: autocoder/pydantic_models.py
#############

CREATE_TASKS_PROMPT = "Create a list of coding tasks, each task can be accomplished by modifying existing files or creating new ones"
create_tasks = action_from_model(
    TaskPlan,
    name="TaskPlan",
    description=CREATE_TASKS_PROMPT,
    stop=True,
    decorators=[traceable(run_type="tool")],
)


class Context(BaseModel):
    instructions: List[str] = Field(
        default=[], description="Instructions relevant to the task."
    )
    queries: List[str] = Field(
        default=[],
        description="List of queries used to extract information from the codebase, encompassing elements such as function names, class names, import statements, variable names, and error messages, all relevant to the task.",
    )
    code_snippets: List[str] = Field(
        default=[],
        description="Search for the code within the codebase.",
    )
    files: List[str] = Field(default=[], description="List of files, e.g. *.py")


CREATE_CONTEXT_PROMPT = "Extract essential details to request more context"
create_context = action_from_model(
    Context,
    name="Context",
    description=CREATE_CONTEXT_PROMPT,
    stop=True,
    decorators=[traceable(run_type="tool")],
)

DIVIDING_LINE = """
###############
This section is divider and not a part of the code.
# {input}
#############
"""
<END OF SNIPPET>
###############
This section is divider and not a part of the code.
Code Snippet From Filepath: autocoder/rag.py
#############

class RepositoryIndex:
    def __init__(self, github_api, github_repository):
        self.github_api = github_api

        content = self.github_api.list_files_in_main_branch()
        files = content.split("\n")[1:]
        self.files = [file for file in files if ".py" in file]

        print(f"Indexing codebase {github_repository}")
        self.documents = []
        for i, file in enumerate(self.files):
            print(f"Indexing {file}")
            text = self.github_api.read_file(file)
            loc = len([line for line in text.split("\n") if bool(line)])
            # TODO: incorporate last_update_time and number_of_commits in metadata
            self.documents.append(
                Document(text=text, metadata={"file": file, "loc": loc})
            )

        code_splitter = CodeSplitter(language="python", chunk_lines_overlap=25)
        service_context = ServiceContext.from_defaults(text_splitter=code_splitter)
        self.index = VectorStoreIndex.from_documents(
            self.documents, service_context=service_context
        )

    def query(self, text: str):
        query_engine = self.index.as_query_engine(
            similarity_top_k=10, response_mode="no_text"
        )
        response = query_engine.query(text)

        nodes = response.source_nodes

        # ranking by score and line of codes
        nodes.sort(key=lambda n: n.score + n.metadata["loc"], reverse=True)
        return nodes
<END OF SNIPPET>
###############
This section is divider and not a part of the code.
Code Snippet From Filepath: autocoder/rag.py
#############

from llama_index import Document, ServiceContext, VectorStoreIndex
from llama_index.node_parser import CodeSplitter
<END OF SNIPPET>
###############
This section is divider and not a part of the code.
Code Snippet From Filepath: main.py
#############

import os

from actionweaver.llms import patch
from bot import AutoCoder
from langchain_community.utilities.github import GitHubAPIWrapper
from langsmith.run_helpers import traceable

from autocoder.rag import RepositoryIndex

assert os.environ["LANGCHAIN_API_KEY"]
assert os.environ["GITHUB_APP_ID"]
assert os.environ["GITHUB_APP_PRIVATE_KEY"]

# If use OpenAI API
assert os.environ["OPENAI_API_KEY"]


os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_TRACING_V2"] = "true"

project_name = "autocoder"
os.environ["LANGCHAIN_PROJECT"] = project_name  # Optional: "default" is used if not set


github_repository = "TengHu/auto_coder"
github_api = GitHubAPIWrapper(
    github_repository=github_repository,
    github_app_id=os.environ["GITHUB_APP_ID"],
    github_app_private_key=os.environ["GITHUB_APP_PRIVATE_KEY"],
)

index = RepositoryIndex(github_api, github_repository)

auto_coder = AutoCoder(github_api, None)
res = auto_coder("What are the open issues?")
print(res)
<END OF SNIPPET>

###############
This section is divider and not a part of the code.
Content From Filepath: autocoder/bot.py
#############

import datetime
import os
import uuid
from typing import List, Union

from actionweaver import action
from actionweaver.utils.tokens import TokenUsageTracker
from langchain_community.utilities.github import GitHubAPIWrapper
from langsmith.run_helpers import traceable
from llama_index import Document, ServiceContext, VectorStoreIndex
from llama_index.node_parser import CodeSplitter
from openai import AzureOpenAI, OpenAI
from pydantic import BaseModel, Field

from autocoder.pydantic_models import create_context, create_tasks
from autocoder.telemetry import trace_client

assert os.environ["MODEL"]
MODEL = os.environ["MODEL"]


DIVIDING_LINE = """
###############
This section is divider and not a part of the code.
{input}
#############

"""


class AutoCoder:
    def __init__(self, github_api, index):
        self.github_api = github_api

        # self.client = trace_client(AzureOpenAI(
        #     azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"),
        #     api_key=os.getenv("AZURE_OPENAI_KEY"),
        #     api_version="2023-10-01-preview"
        # ))
        self.client = trace_client(OpenAI())
        self.messages = [
            {
                "role": "system",
                "content": "You are a coding assistant, you have the capability to assist with code-related tasks and modify files.",
            },
        ]
        self.index = index

        msg = self.create_branch(f"aw_demo_bot")
        print(f"[System] {msg}")

    def __call__(self, input: str):
        self.messages.append({"role": "user", "content": input})

        response = self.client.chat.completions.create(
            model=MODEL,
            messages=self.messages,
            stream=False,
            temperature=0.1,
            actions=[
                self.get_issues,
                self.question_answer,
                self.create_pull_request,
                self.plan_code_change,
            ],
            orch={
                self.plan_code_change.name: None,
                self.create_pull_request.name: None,
            },
            token_usage_tracker=TokenUsageTracker(500),
        )

        content = ""
        try:
            content = response.choices[0].message.content
        except:
            content = str(response)
        self.messages.append({"role": "assistant", "content": content})
        return content

    @traceable(run_type="tool")
    def gather_context(self, input):
        user_prompt = input

        messages = [
            # {
            #     "role": "system",
            #     "content": "You are good at extract information from description",
            # },
            {"role": "user", "content": f"Description: {user_prompt}"},
        ]
        context = create_context.invoke(
            self.client,
            messages=messages,
            model=MODEL,
            stream=False,
            force=True,
        )

        if isinstance(context, list):
            context = context[0]

        index_response = ""
        for query in context.queries + context.instructions:
            nodes = self.index.query(query)
            for node in nodes:
                index_response = (
                    index_response
                    + DIVIDING_LINE.format(
                        input=f"Code Snippet From Filepath: {node.metadata['file']}"
                    )
                    + f"{node.text}\n"
                    + "<END OF SNIPPET>"
                )

        file_response = self.read_files(context.files)
        code_search_response = self.search_code(" ".join(context.code_snippets))

        return (
            "CONTEXT FOR MAKING CODE MODIFICATIONS:\n"
            + index_response
            + "\n"
            + file_response
            + "\n"
            + code_search_response
        )

    @action(name="QuestionAnswer", decorators=[traceable(run_type="tool")])
    def question_answer(self, rewritten_query: str, keywords: List[str]):
        """Answer questions about the codebase"""

        context = self.gather_context(" ".join(keywords))

        messages = [
            {
                "role": "user",
                "content": f"{context} \n###########\n Question: {rewritten_query}",
            }
        ]
        response = self.client.chat.completions.create(
            model=MODEL,
            messages=messages,
            stream=False,
            token_usage_tracker=TokenUsageTracker(500),
        )
        return response

    @action(name="GetIssues", decorators=[traceable(run_type="tool")])
    def get_issues(self):
        """
        Get a list of issues from the GitHub repo.
        """
        response = self.github_api.get_issues()
        response = response.split("\n")
        return eval(response[1]) if len(response) > 1 else []

    @action(name="CreateGitBranch", decorators=[traceable(run_type="tool")])
    def create_branch(self, branch: str):
        """
        Create a new Git branch.
        """
        return self.github_api.create_branch(branch)

    @action(name="CreatePullRequest", decorators=[traceable(run_type="tool")])
    def create_pull_request(self, title: str, description: str):
        """
        Create a new Pull Request in a Git repository.

        Args:
            title (str): The title of the Pull Request.
            description (str): The description of the Pull Request.
        """
        return self.github_api.create_pull_request(pr_query=f"{title}\n {description}")

    @traceable(run_type="tool")
    def read_files(self, files: List[str]) -> List[str]:
        """
        Read the content of multiple files in the GitHub repo

        Args:
            files (List[str]): A list of file paths to be read using the GitHub API.
        """
        response = ""
        for file in files:
            api_response = self.github_api.read_file(file)
            if f"File not found `{file}`" not in api_response:
                response = (
                    response
                    + DIVIDING_LINE.format(input=f"Content From Filepath: {file}")
                    + f"{self.github_api.read_file(file)}\n<END OF FILE>"
                )
        return response

    def rephrase(self, input: str):
        messages = [{"role": "user", "content": f"{input}\n####\nRephrase"}]
        response = self.client.chat.completions.create(
            model=MODEL,
            messages=messages,
            stream=False,
            temperature=0.1,
            token_usage_tracker=TokenUsageTracker(500),
        )
        content = ""
        try:
            content = response.choices[0].message.content
        except:
            content = str(response)
        return content

    @action("PlanCodeChange", stop=True, decorators=[traceable(run_type="tool")])
    def plan_code_change(self, description: str):
        """
        Plan code changes based on a given description.

        This method is designed to handle various types of code alterations such as
        inserting new code, refactoring existing code, replacing segments, or making
        general modifications.
        """
        context = self.gather_context(description)

        user_prompt = f"""
        {context}
        {'#' * 20}
        Description:
        {description}"""

        messages = [{"role": "user", "content": user_prompt}]
        tasks = create_tasks.invoke(
            self.client,
            messages=messages,
            temperature=0.1,
            model=MODEL,
            stream=False,
            force=True,
        )

        if isinstance(tasks, list):
            tasks = tasks[0]
        messages = tasks.execute(self.client, self.github_api, context)

        files_updated = []
        files_created = []
        problems = []
        for msg in messages:
            if "Updated file" in str(msg):
                files_updated.append(msg)
            elif "Created file" in str(msg):
                files_created.append(msg)
            else:
                problems.append(msg)

        return self.rephrase(
            f"""
- New files created: {files_created}
- Existing files updated: {files_updated}
- Problems encountered: {problems}
"""
        )

    def search_code(self, query: str):
        return self.github_api.search_code(query)

<END OF FILE>
Showing top 5 of 10 results:
Filepath: `README.md`
File contents: # auto_coder
<END OF FILE>
Filepath: `main.py`
File contents: import os

from actionweaver.llms import patch
from bot import AutoCoder
from langchain_community.utilities.github import GitHubAPIWrapper
from langsmith.run_helpers import traceable

from autocoder.rag import RepositoryIndex

assert os.environ["LANGCHAIN_API_KEY"]
assert os.environ["GITHUB_APP_ID"]
assert os.environ["GITHUB_APP_PRIVATE_KEY"]

# If use OpenAI API
assert os.environ["OPENAI_API_KEY"]


os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_TRACING_V2"] = "true"

project_name = "autocoder"
os.environ["LANGCHAIN_PROJECT"] = project_name  # Optional: "default" is used if not set


github_repository = "TengHu/auto_coder"
github_api = GitHubAPIWrapper(
    github_repository=github_repository,
    github_app_id=os.environ["GITHUB_APP_ID"],
    github_app_private_key=os.environ["GITHUB_APP_PRIVATE_KEY"],
)

index = RepositoryIndex(github_api, github_repository)

auto_coder = AutoCoder(github_api, None)
res = auto_coder("What are the open issues?")
print(res)

<END OF FILE>
Filepath: `autocoder/bot.py`
File contents: import datetime
import os
import uuid
from typing import List, Union

from actionweaver import action
from actionweaver.utils.tokens import TokenUsageTracker
from langchain_community.utilities.github import GitHubAPIWrapper
from langsmith.run_helpers import traceable
from llama_index import Document, ServiceContext, VectorStoreIndex
from llama_index.node_parser import CodeSplitter
from openai import AzureOpenAI, OpenAI
from pydantic import BaseModel, Field

from autocoder.pydantic_models import create_context, create_tasks
from autocoder.telemetry import trace_client

assert os.environ["MODEL"]
MODEL = os.environ["MODEL"]


DIVIDING_LINE = """
###############
This section is divider and not a part of the code.
{input}
#############

"""


class AutoCoder:
    def __init__(self, github_api, index):
        self.github_api = github_api

        # self.client = trace_client(AzureOpenAI(
        #     azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"),
        #     api_key=os.getenv("AZURE_OPENAI_KEY"),
        #     api_version="2023-10-01-preview"
        # ))
        self.client = trace_client(OpenAI())
        self.messages = [
            {
                "role": "system",
                "content": "You are a coding assistant, you have the capability to assist with code-related tasks and modify files.",
            },
        ]
        self.index = index

        msg = self.create_branch(f"aw_demo_bot")
        print(f"[System] {msg}")

    def __call__(self, input: str):
        self.messages.append({"role": "user", "content": input})

        response = self.client.chat.completions.create(
            model=MODEL,
            messages=self.messages,
            stream=False,
            temperature=0.1,
            actions=[
                self.get_issues,
                self.question_answer,
                self.create_pull_request,
                self.plan_code_change,
            ],
            orch={
                self.plan_code_change.name: None,
                self.create_pull_request.name: None,
            },
            token_usage_tracker=TokenUsageTracker(500),
        )

        content = ""
        try:
            content = response.choices[0].message.content
        except:
            content = str(response)
        self.messages.append({"role": "assistant", "content": content})
        return content

    @traceable(run_type="tool")
    def gather_context(self, input):
        user_prompt = input

        messages = [
            # {
            #     "role": "system",
            #     "content": "You are good at extract information from description",
            # },
            {"role": "user", "content": f"Description: {user_prompt}"},
        ]
        context = create_context.invoke(
            self.client,
            messages=messages,
            model=MODEL,
            stream=False,
            force=True,
        )

        if isinstance(context, list):
            context = context[0]

        index_response = ""
        for query in context.queries + context.instructions:
            nodes = self.index.query(query)
            for node in nodes:
                index_response = (
                    index_response
                    + DIVIDING_LINE.format(
                        input=f"Code Snippet From Filepath: {node.metadata['file']}"
                    )
                    + f"{node.text}\n"
                    + "<END OF SNIPPET>"
                )

        file_response = self.read_files(context.files)
        code_search_response = self.search_code(" ".join(context.code_snippets))

        return (
            "CONTEXT FOR MAKING CODE MODIFICATIONS:\n"
            + index_response
            + "\n"
            + file_response
            + "\n"
            + code_search_response
        )

    @action(name="QuestionAnswer", decorators=[traceable(run_type="tool")])
    def question_answer(self, rewritten_query: str, keywords: List[str]):
        """Answer questions about the codebase"""

        context = self.gather_context(" ".join(keywords))

        messages = [
            {
                "role": "user",
                "content": f"{context} \n###########\n Question: {rewritten_query}",
            }
        ]
        response = self.client.chat.completions.create(
            model=MODEL,
            messages=messages,
            stream=False,
            token_usage_tracker=TokenUsageTracker(500),
        )
        return response

    @action(name="GetIssues", decorators=[traceable(run_type="tool")])
    def get_issues(self):
        """
        Get a list of issues from the GitHub repo.
        """
        response = self.github_api.get_issues()
        response = response.split("\n")
        return eval(response[1]) if len(response) > 1 else []

    @action(name="CreateGitBranch", decorators=[traceable(run_type="tool")])
    def create_branch(self, branch: str):
        """
        Create a new Git branch.
        """
        return self.github_api.create_branch(branch)

    @action(name="CreatePullRequest", decorators=[traceable(run_type="tool")])
    def create_pull_request(self, title: str, description: str):
        """
        Create a new Pull Request in a Git repository.

        Args:
            title (str): The title of the Pull Request.
            description (str): The description of the Pull Request.
        """
        return self.github_api.create_pull_request(pr_query=f"{title}\n {description}")

    @traceable(run_type="tool")
    def read_files(self, files: List[str]) -> List[str]:
        """
        Read the content of multiple files in the GitHub repo

        Args:
            files (List[str]): A list of file paths to be read using the GitHub API.
        """
        response = ""
        for file in files:
            api_response = self.github_api.read_file(file)
            if f"File not found `{file}`" not in api_response:
                response = (
                    response
                    + DIVIDING_LINE.format(input=f"Content From Filepath: {file}")
                    + f"{self.github_api.read_file(file)}\n<END OF FILE>"
                )
        return response

    def rephrase(self, input: str):
        messages = [{"role": "user", "content": f"{input}\n####\nRephrase"}]
        response = self.client.chat.completions.create(
            model=MODEL,
            messages=messages,
            stream=False,
            temperature=0.1,
            token_usage_tracker=TokenUsageTracker(500),
        )
        content = ""
        try:
            content = response.choices[0].message.content
        except:
            content = str(response)
        return content

    @action("PlanCodeChange", stop=True, decorators=[traceable(run_type="tool")])
    def plan_code_change(self, description: str):
        """
        Plan code changes based on a given description.

        This method is designed to handle various types of code alterations such as
        inserting new code, refactoring existing code, replacing segments, or making
        general modifications.
        """
        context = self.gather_context(description)

        user_prompt = f"""
        {context}
        {'#' * 20}
        Description:
        {description}"""

        messages = [{"role": "user", "content": user_prompt}]
        tasks = create_tasks.invoke(
            self.client,
            messages=messages,
            temperature=0.1,
            model=MODEL,
            stream=False,
            force=True,
        )

        if isinstance(tasks, list):
            tasks = tasks[0]
        messages = tasks.execute(self.client, self.github_api, context)

        files_updated = []
        files_created = []
        problems = []
        for msg in messages:
            if "Updated file" in str(msg):
                files_updated.append(msg)
            elif "Created file" in str(msg):
                files_created.append(msg)
            else:
                problems.append(msg)

        return self.rephrase(
            f"""
- New files created: {files_created}
- Existing files updated: {files_updated}
- Problems encountered: {problems}
"""
        )

    def search_code(self, query: str):
        return self.github_api.search_code(query)

<END OF FILE>
Filepath: `autocoder/rag.py`
File contents: from llama_index import Document, ServiceContext, VectorStoreIndex
from llama_index.node_parser import CodeSplitter


class RepositoryIndex:
    def __init__(self, github_api, github_repository):
        self.github_api = github_api

        content = self.github_api.list_files_in_main_branch()
        files = content.split("\n")[1:]
        self.files = [file for file in files if ".py" in file]

        print(f"Indexing codebase {github_repository}")
        self.documents = []
        for i, file in enumerate(self.files):
            print(f"Indexing {file}")
            text = self.github_api.read_file(file)
            loc = len([line for line in text.split("\n") if bool(line)])
            # TODO: incorporate last_update_time and number_of_commits in metadata
            self.documents.append(
                Document(text=text, metadata={"file": file, "loc": loc})
            )

        code_splitter = CodeSplitter(language="python", chunk_lines_overlap=25)
        service_context = ServiceContext.from_defaults(text_splitter=code_splitter)
        self.index = VectorStoreIndex.from_documents(
            self.documents, service_context=service_context
        )

    def query(self, text: str):
        query_engine = self.index.as_query_engine(
            similarity_top_k=10, response_mode="no_text"
        )
        response = query_engine.query(text)

        nodes = response.source_nodes

        # ranking by score and line of codes
        nodes.sort(key=lambda n: n.score + n.metadata["loc"], reverse=True)
        return nodes

<END OF FILE>
Filepath: `notebooks/demo.ipynb`
File contents: {
 "cells": [
  {
   "cell_type": "code",
   "execution_count": 1,
   "id": "73541e6c-f4d8-4ed5-bfee-bbf10d1072e8",
   "metadata": {},
   "outputs": [],
   "source": [
    "%load_ext autoreload\n",
    "%autoreload 2"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 2,
   "id": "4032d086-3eaf-48d3-8adf-5c350afc06d6",
   "metadata": {},
   "outputs": [],
   "source": [
    "import os\n",
    "import sys\n",
    "\n",
    "parent_directory = os.path.abspath('..')\n",
    "sys.path.append(parent_directory)\n",
    "\n",
    "from actionweaver.llms import patch\n",
    "from langchain_community.utilities.github import GitHubAPIWrapper\n",
    "from langsmith.run_helpers import traceable\n",
    "\n",
    "from autocoder.bot import AutoCoder\n",
    "from autocoder.rag import RepositoryIndex"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 3,
   "id": "8cd3dc08-68d8-49a9-95c4-f1d5a1c178d6",
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Indexing codebase TengHu/auto_coder\n",
      "Indexing autocoder/__init__.py\n",
      "Indexing autocoder/bot.py\n",
      "Indexing autocoder/pydantic_models.py\n",
      "Indexing autocoder/rag.py\n",
      "Indexing autocoder/telemetry.py\n",
      "Indexing main.py\n"
     ]
    }
   ],
   "source": [
    "github_repository = \"TengHu/auto_coder\"\n",
    "github_api = GitHubAPIWrapper(\n",
    "    github_repository=github_repository,\n",
    "    github_app_id=os.environ[\"GITHUB_APP_ID\"],\n",
    "    github_app_private_key=os.environ[\"GITHUB_APP_PRIVATE_KEY\"],\n",
    ")\n",
    "\n",
    "index = RepositoryIndex(github_api, github_repository)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "23d62a18-5426-450b-ba45-e63bbf2dc3c6",
   "metadata": {},
   "outputs": [],
   "source": []
  },
  {
   "cell_type": "code",
   "execution_count": 8,
   "id": "5137b4db-2208-4faf-a479-17076da18e9a",
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "[System] Branch 'aw_demo_bot_v8' created successfully, and set as current active branch.\n"
     ]
    }
   ],
   "source": [
    "auto_coder = AutoCoder(github_api, index)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 9,
   "id": "bad6fbc4-e66a-408e-867d-b617b6703275",
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "[\"- No new files were created.\\n- The following existing files received updates: \\n  - 'Updated file autocoder/bot.py' (mentioned multiple times, possibly indicating multiple update attempts)\\n  - 'Updated file autocoder/rag.py'\\n- Issues encountered included:\\n  - Updates to file content failed because the expected old content was not located. It might be beneficial to utilize the read_file action to retrieve the latest file contents before attempting updates. This issue was noted twice, suggesting repeated problems with updating file content.\"]\n"
     ]
    }
   ],
   "source": [
    "res = auto_coder(\"remove unused imports in autocoder/bot.py\")\n",
    "print(res)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 7,
   "id": "73751d5b-cfaa-4885-8445-746a551da29c",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "'The pull request has been successfully created with the number 31. It includes the changes to reorder the imports in `autocoder/bot.py` to follow PEP 8 guidelines for better code readability and standardization.'"
      ]
     },
     "execution_count": 7,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "auto_coder(\"create a pr\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 11,
   "id": "4d16a64d-de27-430f-b742-7c045af08813",
   "metadata": {},
   "outputs": [],
   "source": [
    "nodes = index.query(\"AutoCoder\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 21,
   "id": "e5503aa5-b3af-400d-9d6a-79e05e12a372",
   "metadata": {},
   "outputs": [],
   "source": [
    "nodes.sort(key=lambda n: n.score + n.metadata['loc'], reverse=True)\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 20,
   "id": "cb77a698-4ea1-4cda-b0e8-56bf10bea428",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "221"
      ]
     },
     "execution_count": 20,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "nodes[-1].metadata['loc']"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 22,
   "id": "df5dbb4c-b934-48a1-9298-32a4f14bae59",
   "metadata": {},
   "outputs": [],
   "source": [
    "res = auto_coder.search_code(\"AutoCoder\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 23,
   "id": "fc9d3d0e-232b-4f61-98a1-7eab421d7e37",
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Showing top 4 of 4 results:\n",
      "Filepath: `autocoder/bot.py`\n",
      "File contents: import datetime\n",
      "import os\n",
      "import uuid\n",
      "from typing import List, Union\n",
      "\n",
      "from actionweaver import action\n",
      "from actionweaver.utils.tokens import TokenUsageTracker\n",
      "from langchain_community.utilities.github import GitHubAPIWrapper\n",
      "from langsmith.run_helpers import traceable\n",
      "from llama_index import Document, ServiceContext, VectorStoreIndex\n",
      "from llama_index.node_parser import CodeSplitter\n",
      "from openai import AzureOpenAI, OpenAI\n",
      "from pydantic import BaseModel, Field\n",
      "\n",
      "from autocoder.pydantic_models import create_context, create_tasks\n",
      "from autocoder.telemetry import trace_client\n",
      "\n",
      "assert os.environ[\"MODEL\"]\n",
      "MODEL = os.environ[\"MODEL\"]\n",
      "\n",
      "\n",
      "DIVIDING_LINE = \"\"\"\n",
      "###############\n",
      "This section is divider and not a part of the code.\n",
      "{input}\n",
      "#############\n",
      "\n",
      "\"\"\"\n",
      "\n",
      "\n",
      "class AutoCoder:\n",
      "    def __init__(self, github_api, index):\n",
      "        self.github_api = github_api\n",
      "\n",
      "        # self.client = trace_client(AzureOpenAI(\n",
      "        #     azure_endpoint = os.getenv(\"AZURE_OPENAI_ENDPOINT\"),\n",
      "        #     api_key=os.getenv(\"AZURE_OPENAI_KEY\"),\n",
      "        #     api_version=\"2023-10-01-preview\"\n",
      "        # ))\n",
      "        self.client = trace_client(OpenAI())\n",
      "        self.messages = [\n",
      "            {\n",
      "                \"role\": \"system\",\n",
      "                \"content\": \"You are a coding assistant, you have the capability to assist with code-related tasks and modify files.\",\n",
      "            },\n",
      "        ]\n",
      "        self.index = index\n",
      "\n",
      "        msg = self.create_branch(f\"aw_demo_bot\")\n",
      "        print(f\"[System] {msg}\")\n",
      "\n",
      "    def __call__(self, input: str):\n",
      "        self.messages.append({\"role\": \"user\", \"content\": input})\n",
      "\n",
      "        response = self.client.chat.completions.create(\n",
      "            model=MODEL,\n",
      "            messages=self.messages,\n",
      "            stream=False,\n",
      "            temperature=0.1,\n",
      "            actions=[\n",
      "                self.get_issues,\n",
      "                self.question_answer,\n",
      "                self.create_pull_request,\n",
      "                self.plan_code_change,\n",
      "            ],\n",
      "            orch={\n",
      "                self.plan_code_change.name: None,\n",
      "                self.create_pull_request.name: None,\n",
      "            },\n",
      "            token_usage_tracker=TokenUsageTracker(500),\n",
      "        )\n",
      "\n",
      "        content = \"\"\n",
      "        try:\n",
      "            content = response.choices[0].message.content\n",
      "        except:\n",
      "            content = str(response)\n",
      "        self.messages.append({\"role\": \"assistant\", \"content\": content})\n",
      "        return content\n",
      "\n",
      "    @traceable(run_type=\"tool\")\n",
      "    def gather_context(self, input, additional_params=None):\n",
      "        user_prompt = input\n",
      "\n",
      "        messages = [\n",
      "            # {\n",
      "            #     \"role\": \"system\",\n",
      "            #     \"content\": \"You are good at extract information from description\",\n",
      "            # },\n",
      "            {\"role\": \"user\", \"content\": f\"Description: {user_prompt}\"},\n",
      "        ]\n",
      "        context = create_context.invoke(\n",
      "            self.client,\n",
      "            messages=messages,\n",
      "            model=MODEL,\n",
      "            stream=False,\n",
      "            force=True,\n",
      "        )\n",
      "\n",
      "        if isinstance(context, list):\n",
      "            context = context[0]\n",
      "\n",
      "        index_response = \"\"\n",
      "        for query in context.semantic_queries + context.instructions:\n",
      "            nodes = self.index.query(query)\n",
      "            for node in nodes:\n",
      "                index_response = (\n",
      "                    index_response\n",
      "                    + DIVIDING_LINE.format(\n",
      "                        input=f\"Code Snippet From File: {node.metadata['file']}\"\n",
      "                    )\n",
      "                    + f\"{node.text}\"\n",
      "                )\n",
      "\n",
      "        file_response = self.read_files(context.files)\n",
      "\n",
      "        return (\n",
      "            \"CONTEXT FOR MAKING CODE MODIFICATIONS:\\n\"\n",
      "            + index_response\n",
      "            + \"\\n\"\n",
      "            + file_response\n",
      "        )\n",
      "\n",
      "    @action(name=\"QuestionAnswer\", decorators=[traceable(run_type=\"tool\")])\n",
      "    def question_answer(self, rewritten_query: str, keywords: List[str]):\n",
      "        \"\"\"Answer questions about the codebase\"\"\"\n",
      "\n",
      "        context = self.gather_context(description=\" \".join(keywords))\n",
      "\n",
      "        messages = [\n",
      "            {\n",
      "                \"role\": \"user\",\n",
      "                \"content\": f\"{context} \\n###########\\n Question: {rewritten_query}\",\n",
      "            }\n",
      "        ]\n",
      "        response = self.client.chat.completions.create(\n",
      "            model=MODEL,\n",
      "            messages=messages,\n",
      "            stream=False,\n",
      "            token_usage_tracker=TokenUsageTracker(500),\n",
      "        )\n",
      "        return response\n",
      "\n",
      "    @action(name=\"GetIssues\", decorators=[traceable(run_type=\"tool\")])\n",
      "    def get_issues(self):\n",
      "        \"\"\"\n",
      "        Get a list of issues from the GitHub repo.\n",
      "        \"\"\"\n",
      "        response = self.github_api.get_issues()\n",
      "        response = response.split(\"\\n\")\n",
      "        return eval(response[1]) if len(response) > 1 else []\n",
      "\n",
      "    @action(name=\"CreateGitBranch\", decorators=[traceable(run_type=\"tool\")])\n",
      "    def create_branch(self, branch: str):\n",
      "        \"\"\"\n",
      "        Create a new Git branch.\n",
      "        \"\"\"\n",
      "        return self.github_api.create_branch(branch)\n",
      "\n",
      "    @action(name=\"CreatePullRequest\", decorators=[traceable(run_type=\"tool\")])\n",
      "    def create_pull_request(self, title: str, description: str):\n",
      "        \"\"\"\n",
      "        Create a new Pull Request in a Git repository.\n",
      "\n",
      "        Args:\n",
      "            title (str): The title of the Pull Request.\n",
      "            description (str): The description of the Pull Request.\n",
      "        \"\"\"\n",
      "        return self.github_api.create_pull_request(pr_query=f\"{title}\\n {description}\")\n",
      "\n",
      "    @traceable(run_type=\"tool\")\n",
      "    def read_files(self, files: List[str]) -> List[str]:\n",
      "        \"\"\"\n",
      "        Read the content of multiple files in the GitHub repo\n",
      "\n",
      "        Args:\n",
      "            files (List[str]): A list of file paths to be read using the GitHub API.\n",
      "        \"\"\"\n",
      "        response = \"\"\n",
      "        for file in files:\n",
      "            api_response = self.github_api.read_file(file)\n",
      "            if f\"File not found `{file}`\" not in api_response:\n",
      "                response = (\n",
      "                    response\n",
      "                    + DIVIDING_LINE.format(input=f\"{file} full content:\")\n",
      "                    + f\"{self.github_api.read_file(file)}\\n\"\n",
      "                )\n",
      "        return response\n",
      "\n",
      "    def rephrase(self, input: str):\n",
      "        messages = [{\"role\": \"user\", \"content\": f\"{input}\\n####\\nRephrase\"}]\n",
      "        response = self.client.chat.completions.create(\n",
      "            model=MODEL,\n",
      "            messages=messages,\n",
      "            stream=False,\n",
      "            temperature=0.1,\n",
      "            token_usage_tracker=TokenUsageTracker(500),\n",
      "        )\n",
      "        content = \"\"\n",
      "        try:\n",
      "            content = response.choices[0].message.content\n",
      "        except:\n",
      "            content = str(response)\n",
      "        return content\n",
      "\n",
      "    @action(\"PlanCodeChange\", stop=True, decorators=[traceable(run_type=\"tool\")])\n",
      "    def plan_code_change(self, description: str):\n",
      "        \"\"\"\n",
      "        Plan code changes based on a given description.\n",
      "\n",
      "        This method is designed to handle various types of code alterations such as\n",
      "        inserting new code, refactoring existing code, replacing segments, or making\n",
      "        general modifications.\n",
      "        \"\"\"\n",
      "        context = self.gather_context(description=description)\n",
      "\n",
      "        user_prompt = f\"\"\"\n",
      "        {context}\n",
      "        {'#' * 20}\n",
      "        Description:\n",
      "        {description}\"\"\"\n",
      "\n",
      "        messages = [{\"role\": \"user\", \"content\": user_prompt}]\n",
      "        tasks = create_tasks.invoke(\n",
      "            self.client,\n",
      "            messages=messages,\n",
      "            temperature=0.1,\n",
      "            model=MODEL,\n",
      "            stream=False,\n",
      "            force=True,\n",
      "        )\n",
      "\n",
      "        if isinstance(tasks, list):\n",
      "            tasks = tasks[0]\n",
      "        messages = tasks.execute(self.client, self.github_api, context)\n",
      "\n",
      "        files_updated = []\n",
      "        files_created = []\n",
      "        problems = []\n",
      "        for msg in messages:\n",
      "            if \"Updated file\" in str(msg):\n",
      "                files_updated.append(msg)\n",
      "            elif \"Created file\" in str(msg):\n",
      "                files_created.append(msg)\n",
      "            else:\n",
      "                problems.append(msg)\n",
      "\n",
      "        return self.rephrase(\n",
      "            f\"\"\"\n",
      "- New files created: {files_created}\n",
      "- Existing files updated: {files_updated}\n",
      "- Problems encountered: {problems}\n",
      "\"\"\"\n",
      "        )\n",
      "\n",
      "    @action(name=\"SearchCode\", decorators=[traceable(run_type=\"tool\")])\n",
      "    def search_code(self, query: str):\n",
      "        return self.github_api.search_code(query)\n",
      "\n",
      "<END OF FILE>\n",
      "Filepath: `autocoder/telemetry.py`\n",
      "File contents: import os\n",
      "\n",
      "from actionweaver.llms import patch\n",
      "from langsmith.run_helpers import traceable\n",
      "\n",
      "os.environ[\"LANGCHAIN_ENDPOINT\"] = \"https://api.smith.langchain.com\"\n",
      "os.environ[\"LANGCHAIN_TRACING_V2\"] = \"true\"\n",
      "project_name = \"autocoder\"\n",
      "os.environ[\"LANGCHAIN_PROJECT\"] = project_name  # Optional: \"default\" is used if not set\n",
      "\n",
      "assert os.environ[\"LANGCHAIN_API_KEY\"]\n",
      "\n",
      "\n",
      "def trace_client(client):\n",
      "    client.chat.completions.create = traceable(name=\"llm_call\", run_type=\"llm\")(\n",
      "        client.chat.completions.create\n",
      "    )\n",
      "    client = patch(client)\n",
      "    client.chat.completions.create = traceable(\n",
      "        name=\"chat_completion_create\", run_type=\"llm\"\n",
      "    )(client.chat.completions.create)\n",
      "    return client\n",
      "\n",
      "<END OF FILE>\n",
      "Filepath: `main.py`\n",
      "File contents: import os\n",
      "\n",
      "from actionweaver.llms import patch\n",
      "from bot import AutoCoder\n",
      "from langchain_community.utilities.github import GitHubAPIWrapper\n",
      "from langsmith.run_helpers import traceable\n",
      "\n",
      "from autocoder.rag import RepositoryIndex\n",
      "\n",
      "assert os.environ[\"LANGCHAIN_API_KEY\"]\n",
      "assert os.environ[\"GITHUB_APP_ID\"]\n",
      "assert os.environ[\"GITHUB_APP_PRIVATE_KEY\"]\n",
      "\n",
      "# If use OpenAI API\n",
      "assert os.environ[\"OPENAI_API_KEY\"]\n",
      "\n",
      "\n",
      "os.environ[\"LANGCHAIN_ENDPOINT\"] = \"https://api.smith.langchain.com\"\n",
      "os.environ[\"LANGCHAIN_TRACING_V2\"] = \"true\"\n",
      "\n",
      "project_name = \"autocoder\"\n",
      "os.environ[\"LANGCHAIN_PROJECT\"] = project_name  # Optional: \"default\" is used if not set\n",
      "\n",
      "\n",
      "github_repository = \"TengHu/auto_coder\"\n",
      "github_api = GitHubAPIWrapper(\n",
      "    github_repository=github_repository,\n",
      "    github_app_id=os.environ[\"GITHUB_APP_ID\"],\n",
      "    github_app_private_key=os.environ[\"GITHUB_APP_PRIVATE_KEY\"],\n",
      ")\n",
      "\n",
      "index = RepositoryIndex(github_api, github_repository)\n",
      "\n",
      "auto_coder = AutoCoder(github_api, None)\n",
      "res = auto_coder(\"What are the open issues?\")\n",
      "print(res)\n",
      "\n",
      "<END OF FILE>\n",
      "Filepath: `notebooks/demo.ipynb`\n",
      "File contents: {\n",
      " \"cells\": [\n",
      "  {\n",
      "   \"cell_type\": \"code\",\n",
      "   \"execution_count\": 1,\n",
      "   \"id\": \"73541e6c-f4d8-4ed5-bfee-bbf10d1072e8\",\n",
      "   \"metadata\": {},\n",
      "   \"outputs\": [],\n",
      "   \"source\": [\n",
      "    \"%load_ext autoreload\\n\",\n",
      "    \"%autoreload 2\"\n",
      "   ]\n",
      "  },\n",
      "  {\n",
      "   \"cell_type\": \"code\",\n",
      "   \"execution_count\": 2,\n",
      "   \"id\": \"4032d086-3eaf-48d3-8adf-5c350afc06d6\",\n",
      "   \"metadata\": {},\n",
      "   \"outputs\": [],\n",
      "   \"source\": [\n",
      "    \"import os\\n\",\n",
      "    \"import sys\\n\",\n",
      "    \"\\n\",\n",
      "    \"parent_directory = os.path.abspath('..')\\n\",\n",
      "    \"sys.path.append(parent_directory)\\n\",\n",
      "    \"\\n\",\n",
      "    \"from actionweaver.llms import patch\\n\",\n",
      "    \"from langchain_community.utilities.github import GitHubAPIWrapper\\n\",\n",
      "    \"from langsmith.run_helpers import traceable\\n\",\n",
      "    \"\\n\",\n",
      "    \"from autocoder.bot import AutoCoder\\n\",\n",
      "    \"from autocoder.rag import RepositoryIndex\"\n",
      "   ]\n",
      "  },\n",
      "  {\n",
      "   \"cell_type\": \"code\",\n",
      "   \"execution_count\": 3,\n",
      "   \"id\": \"8cd3dc08-68d8-49a9-95c4-f1d5a1c178d6\",\n",
      "   \"metadata\": {},\n",
      "   \"outputs\": [\n",
      "    {\n",
      "     \"name\": \"stdout\",\n",
      "     \"output_type\": \"stream\",\n",
      "     \"text\": [\n",
      "      \"Indexing codebase TengHu/auto_coder\\n\",\n",
      "      \"Indexing autocoder/bot.py\\n\",\n",
      "      \"Indexing autocoder/rag.py\\n\",\n",
      "      \"Indexing autocoder/telemetry.py\\n\",\n",
      "      \"Indexing main.py\\n\",\n",
      "      \"Indexing pydantic_models.py\\n\"\n",
      "     ]\n",
      "    }\n",
      "   ],\n",
      "   \"source\": [\n",
      "    \"github_repository = \\\"TengHu/auto_coder\\\"\\n\",\n",
      "    \"github_api = GitHubAPIWrapper(\\n\",\n",
      "    \"    github_repository=github_repository,\\n\",\n",
      "    \"    github_app_id=os.environ[\\\"GITHUB_APP_ID\\\"],\\n\",\n",
      "    \"    github_app_private_key=os.environ[\\\"GITHUB_APP_PRIVATE_KEY\\\"],\\n\",\n",
      "    \")\\n\",\n",
      "    \"\\n\",\n",
      "    \"index = RepositoryIndex(github_api, github_repository)\"\n",
      "   ]\n",
      "  },\n",
      "  {\n",
      "   \"cell_type\": \"code\",\n",
      "   \"execution_count\": null,\n",
      "   \"id\": \"23d62a18-5426-450b-ba45-e63bbf2dc3c6\",\n",
      "   \"metadata\": {},\n",
      "   \"outputs\": [],\n",
      "   \"source\": []\n",
      "  },\n",
      "  {\n",
      "   \"cell_type\": \"code\",\n",
      "   \"execution_count\": 4,\n",
      "   \"id\": \"5137b4db-2208-4faf-a479-17076da18e9a\",\n",
      "   \"metadata\": {},\n",
      "   \"outputs\": [\n",
      "    {\n",
      "     \"name\": \"stdout\",\n",
      "     \"output_type\": \"stream\",\n",
      "     \"text\": [\n",
      "      \"[System] Branch 'aw_demo_bot_v5' created successfully, and set as current active branch.\\n\"\n",
      "     ]\n",
      "    }\n",
      "   ],\n",
      "   \"source\": [\n",
      "    \"auto_coder = AutoCoder(github_api, index)\"\n",
      "   ]\n",
      "  },\n",
      "  {\n",
      "   \"cell_type\": \"code\",\n",
      "   \"execution_count\": 6,\n",
      "   \"id\": \"bad6fbc4-e66a-408e-867d-b617b6703275\",\n",
      "   \"metadata\": {},\n",
      "   \"outputs\": [\n",
      "    {\n",
      "     \"name\": \"stdout\",\n",
      "     \"output_type\": \"stream\",\n",
      "     \"text\": [\n",
      "      \"[\\\"Sure thing! Here's a more user-friendly version:\\\\n\\\\n- No new files have been added.\\\\n- We've made updates to the following files: 'telemetry.py' and 'bot.py' within the 'autocoder' folder.\\\\n- We didn't run into any issues along the way.\\\"]\\n\"\n",
      "     ]\n",
      "    }\n",
      "   ],\n",
      "   \"source\": [\n",
      "    \"res = auto_coder(\\\"Move trace_client function to bot.py.\\\")\\n\",\n",
      "    \"print(res)\"\n",
      "   ]\n",
      "  },\n",
      "  {\n",
      "   \"cell_type\": \"code\",\n",
      "   \"execution_count\": 7,\n",
      "   \"id\": \"73751d5b-cfaa-4885-8445-746a551da29c\",\n",
      "   \"metadata\": {},\n",
      "   \"outputs\": [\n",
      "    {\n",
      "     \"data\": {\n",
      "      \"text/plain\": [\n",
      "       \"'The pull request has been successfully created with the title \\\"Move trace_client Function to bot.py\\\" and the description \\\"This PR moves the trace_client function from its current location to bot.py for better organization and code structure.\\\" It is PR number 27.'\"\n",
      "      ]\n",
      "     },\n",
      "     \"execution_count\": 7,\n",
      "     \"metadata\": {},\n",
      "     \"output_type\": \"execute_result\"\n",
      "    }\n",
      "   ],\n",
      "   \"source\": [\n",
      "    \"auto_coder(\\\"create a pr\\\")\"\n",
      "   ]\n",
      "  },\n",
      "  {\n",
      "   \"cell_type\": \"code\",\n",
      "   \"execution_count\": null,\n",
      "   \"id\": \"4d16a64d-de27-430f-b742-7c045af08813\",\n",
      "   \"metadata\": {},\n",
      "   \"outputs\": [],\n",
      "   \"source\": []\n",
      "  },\n",
      "  {\n",
      "   \"cell_type\": \"code\",\n",
      "   \"execution_count\": null,\n",
      "   \"id\": \"e5503aa5-b3af-400d-9d6a-79e05e12a372\",\n",
      "   \"metadata\": {},\n",
      "   \"outputs\": [],\n",
      "   \"source\": []\n",
      "  },\n",
      "  {\n",
      "   \"cell_type\": \"code\",\n",
      "   \"execution_count\": 29,\n",
      "   \"id\": \"26a71e4f-054c-46d7-8a43-66b07fed9971\",\n",
      "   \"metadata\": {},\n",
      "   \"outputs\": [],\n",
      "   \"source\": [\n",
      "    \"from openai.types.chat.chat_completion_message import ChatCompletionMessage\\n\",\n",
      "    \"from openai.types.chat.chat_completion_message_tool_call import (\\n\",\n",
      "    \"    ChatCompletionMessageToolCall,\\n\",\n",
      "    \"Function\\n\",\n",
      "    \")\\n\",\n",
      "    \"messages = [{'role': 'system',\\n\",\n",
      "    \"  'content': 'You are a coding assistant, you have the capability to assist with code-related tasks and modify files.'},\\n\",\n",
      "    \" {'role': 'user', 'content': 'Move trace_client function to bot.py.'},\\n\",\n",
      "    \" ChatCompletionMessage(content=None, role='assistant', tool_calls=[ChatCompletionMessageToolCall(id='call_9f2agXdE5nnUl1X04XkYHjeQ', function=Function(arguments='{\\\"description\\\":\\\"Move the \\\\'trace_client\\\\' function from its current location to the \\\\'bot.py\\\\' file.\\\"}', name='PlanCodeChange'), type='function')]),\\n\",\n",
      "    \" {'tool_call_id': 'call_9f2agXdE5nnUl1X04XkYHjeQ',\\n\",\n",
      "    \"  'role': 'tool',\\n\",\n",
      "    \"  'name': 'PlanCodeChange',\\n\",\n",
      "    \"  'content': \\\"- New files created: []\\\\n        - Existing files updated: [['Updated file autocoder/bot.py', 'Updated file autocoder/telemetry.py']]\\\\n        - Problems encountered: []\\\\n        ###\\\\n    Ignore others,    Make the previous message more user-friendly\\\\n        \\\"}\\n\",\n",
      "    \"           ]\"\n",
      "   ]\n",
      "  },\n",
      "  {\n",
      "   \"cell_type\": \"code\",\n",
      "   \"execution_count\": 30,\n",
      "   \"id\": \"a1db6eea-8727-4b79-b971-217875e64fc4\",\n",
      "   \"metadata\": {},\n",
      "   \"outputs\": [],\n",
      "   \"source\": [\n",
      "    \"from openai import OpenAI\\n\",\n",
      "    \"\\n\",\n",
      "    \"\\n\",\n",
      "    \"client = OpenAI()\\n\",\n",
      "    \"response = client.chat.completions.create(\\n\",\n",
      "    \"            model=\\\"gpt-4-1106-preview\\\",\\n\",\n",
      "    \"            messages=messages,\\n\",\n",
      "    \"            stream=False,\\n\",\n",
      "    \"        )\"\n",
      "   ]\n",
      "  },\n",
      "  {\n",
      "   \"cell_type\": \"code\",\n",
      "   \"execution_count\": 31,\n",
      "   \"id\": \"6933b5be-f2cb-42c8-8f10-e1ce1f89761e\",\n",
      "   \"metadata\": {},\n",
      "   \"outputs\": [\n",
      "    {\n",
      "     \"name\": \"stdout\",\n",
      "     \"output_type\": \"stream\",\n",
      "     \"text\": [\n",
      "      \"ChatCompletion(id='chatcmpl-8h2sHwJwIR8SOkWxw2ypBGd80Uyom', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='I can assist you with moving the `trace_client` function to `bot.py`, but I need access to the source code or need to know where the current `trace_client` function is located. Can you provide the current code file that contains the `trace_client` function or tell me where to find it?', role='assistant', function_call=None, tool_calls=None))], created=1705269845, model='gpt-4-1106-preview', object='chat.completion', system_fingerprint='fp_168383a679', usage=CompletionUsage(completion_tokens=63, prompt_tokens=135, total_tokens=198))\\n\"\n",
      "     ]\n",
      "    }\n",
      "   ],\n",
      "   \"source\": [\n",
      "    \"print (response)\"\n",
      "   ]\n",
      "  },\n",
      "  {\n",
      "   \"cell_type\": \"code\",\n",
      "   \"execution_count\": 32,\n",
      "   \"id\": \"09cb5b10-eda9-46e0-994d-60b2c5f86fb7\",\n",
      "   \"metadata\": {},\n",
      "   \"outputs\": [\n",
      "    {\n",
      "     \"name\": \"stdout\",\n",
      "     \"output_type\": \"stream\",\n",
      "     \"text\": [\n",
      "      \"I can assist you with moving the `trace_client` function to `bot.py`, but I need access to the source code or need to know where the current `trace_client` function is located. Can you provide the current code file that contains the `trace_client` function or tell me where to find it?\\n\"\n",
      "     ]\n",
      "    }\n",
      "   ],\n",
      "   \"source\": [\n",
      "    \"print (response.choices[0].message.content)\"\n",
      "   ]\n",
      "  },\n",
      "  {\n",
      "   \"cell_type\": \"code\",\n",
      "   \"execution_count\": 17,\n",
      "   \"id\": \"50c37b99-db67-4367-8771-a284e4dfab57\",\n",
      "   \"metadata\": {},\n",
      "   \"outputs\": [\n",
      "    {\n",
      "     \"name\": \"stdout\",\n",
      "     \"output_type\": \"stream\",\n",
      "     \"text\": [\n",
      "      \"You are a coding assistant, you have the capability to assist with code-related tasks and modify files.\\n\"\n",
      "     ]\n",
      "    }\n",
      "   ],\n",
      "   \"source\": [\n",
      "    \"print (messages[0]['content'])\"\n",
      "   ]\n",
      "  },\n",
      "  {\n",
      "   \"cell_type\": \"code\",\n",
      "   \"execution_count\": 18,\n",
      "   \"id\": \"df5b7a06-951b-4cc2-b7d2-cbbc864410b2\",\n",
      "   \"metadata\": {},\n",
      "   \"outputs\": [\n",
      "    {\n",
      "     \"data\": {\n",
      "      \"text/plain\": [\n",
      "       \"[{'role': 'system',\\n\",\n",
      "       \"  'content': 'You are a coding assistant, you have the capability to assist with code-related tasks and modify files.'},\\n\",\n",
      "       \" {'role': 'user', 'content': 'Move trace_client function to bot.py.'},\\n\",\n",
      "       \" ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_9f2agXdE5nnUl1X04XkYHjeQ', function=Function(arguments='{\\\"description\\\":\\\"Move the \\\\'trace_client\\\\' function from its current location to the \\\\'bot.py\\\\' file.\\\"}', name='PlanCodeChange'), type='function')]),\\n\",\n",
      "       \" {'tool_call_id': 'call_9f2agXdE5nnUl1X04XkYHjeQ',\\n\",\n",
      "       \"  'role': 'tool',\\n\",\n",
      "       \"  'name': 'PlanCodeChange',\\n\",\n",
      "       \"  'content': \\\"I've read code and attempted to create code changes. The outcome of these code changes is as follows:\\\\n        - New files created: []\\\\n        - Existing files updated: [['Updated file autocoder/bot.py', 'Updated file autocoder/telemetry.py']]\\\\n        - Problems encountered: []\\\\n        ###\\\\n        Make the previous message more user-friendly\\\\n        \\\"}]\"\n",
      "      ]\n",
      "     },\n",
      "     \"execution_count\": 18,\n",
      "     \"metadata\": {},\n",
      "     \"output_type\": \"execute_result\"\n",
      "    }\n",
      "   ],\n",
      "   \"source\": [\n",
      "    \"messages\"\n",
      "   ]\n",
      "  },\n",
      "  {\n",
      "   \"cell_type\": \"code\",\n",
      "   \"execution_count\": null,\n",
      "   \"id\": \"2939b91a-fe15-4338-a52a-d4a7da7a1aca\",\n",
      "   \"metadata\": {},\n",
      "   \"outputs\": [],\n",
      "   \"source\": []\n",
      "  },\n",
      "  {\n",
      "   \"cell_type\": \"code\",\n",
      "   \"execution_count\": null,\n",
      "   \"id\": \"412d9f50-be8d-4f9c-b166-59ba5f876748\",\n",
      "   \"metadata\": {},\n",
      "   \"outputs\": [],\n",
      "   \"source\": []\n",
      "  },\n",
      "  {\n",
      "   \"cell_type\": \"code\",\n",
      "   \"execution_count\": null,\n",
      "   \"id\": \"dcf75302-039f-44d8-a7da-791176d6db85\",\n",
      "   \"metadata\": {},\n",
      "   \"outputs\": [],\n",
      "   \"source\": []\n",
      "  }\n",
      " ],\n",
      " \"metadata\": {\n",
      "  \"kernelspec\": {\n",
      "   \"display_name\": \"Python 3 (ipykernel)\",\n",
      "   \"language\": \"python\",\n",
      "   \"name\": \"python3\"\n",
      "  },\n",
      "  \"language_info\": {\n",
      "   \"codemirror_mode\": {\n",
      "    \"name\": \"ipython\",\n",
      "    \"version\": 3\n",
      "   },\n",
      "   \"file_extension\": \".py\",\n",
      "   \"mimetype\": \"text/x-python\",\n",
      "   \"name\": \"python\",\n",
      "   \"nbconvert_exporter\": \"python\",\n",
      "   \"pygments_lexer\": \"ipython3\",\n",
      "   \"version\": \"3.9.18\"\n",
      "  }\n",
      " },\n",
      " \"nbformat\": 4,\n",
      " \"nbformat_minor\": 5\n",
      "}\n",
      "\n",
      "<END OF FILE>\n"
     ]
    }
   ],
   "source": [
    "print (res)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 29,
   "id": "26a71e4f-054c-46d7-8a43-66b07fed9971",
   "metadata": {},
   "outputs": [],
   "source": [
    "from openai.types.chat.chat_completion_message import ChatCompletionMessage\n",
    "from openai.types.chat.chat_completion_message_tool_call import (\n",
    "    ChatCompletionMessageToolCall,\n",
    "Function\n",
    ")\n",
    "messages = [{'role': 'system',\n",
    "  'content': 'You are a coding assistant, you have the capability to assist with code-related tasks and modify files.'},\n",
    " {'role': 'user', 'content': 'Move trace_client function to bot.py.'},\n",
    " ChatCompletionMessage(content=None, role='assistant', tool_calls=[ChatCompletionMessageToolCall(id='call_9f2agXdE5nnUl1X04XkYHjeQ', function=Function(arguments='{\"description\":\"Move the \\'trace_client\\' function from its current location to the \\'bot.py\\' file.\"}', name='PlanCodeChange'), type='function')]),\n",
    " {'tool_call_id': 'call_9f2agXdE5nnUl1X04XkYHjeQ',\n",
    "  'role': 'tool',\n",
    "  'name': 'PlanCodeChange',\n",
    "  'content': \"- New files created: []\\n        - Existing files updated: [['Updated file autocoder/bot.py', 'Updated file autocoder/telemetry.py']]\\n        - Problems encountered: []\\n        ###\\n    Ignore others,    Make the previous message more user-friendly\\n        \"}\n",
    "           ]"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 30,
   "id": "a1db6eea-8727-4b79-b971-217875e64fc4",
   "metadata": {},
   "outputs": [],
   "source": [
    "from openai import OpenAI\n",
    "\n",
    "\n",
    "client = OpenAI()\n",
    "response = client.chat.completions.create(\n",
    "            model=\"gpt-4-1106-preview\",\n",
    "            messages=messages,\n",
    "            stream=False,\n",
    "        )"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 31,
   "id": "6933b5be-f2cb-42c8-8f10-e1ce1f89761e",
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "ChatCompletion(id='chatcmpl-8h2sHwJwIR8SOkWxw2ypBGd80Uyom', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='I can assist you with moving the `trace_client` function to `bot.py`, but I need access to the source code or need to know where the current `trace_client` function is located. Can you provide the current code file that contains the `trace_client` function or tell me where to find it?', role='assistant', function_call=None, tool_calls=None))], created=1705269845, model='gpt-4-1106-preview', object='chat.completion', system_fingerprint='fp_168383a679', usage=CompletionUsage(completion_tokens=63, prompt_tokens=135, total_tokens=198))\n"
     ]
    }
   ],
   "source": [
    "print (response)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 32,
   "id": "09cb5b10-eda9-46e0-994d-60b2c5f86fb7",
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "I can assist you with moving the `trace_client` function to `bot.py`, but I need access to the source code or need to know where the current `trace_client` function is located. Can you provide the current code file that contains the `trace_client` function or tell me where to find it?\n"
     ]
    }
   ],
   "source": [
    "print (response.choices[0].message.content)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 17,
   "id": "50c37b99-db67-4367-8771-a284e4dfab57",
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "You are a coding assistant, you have the capability to assist with code-related tasks and modify files.\n"
     ]
    }
   ],
   "source": [
    "print (messages[0]['content'])"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 18,
   "id": "df5b7a06-951b-4cc2-b7d2-cbbc864410b2",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "[{'role': 'system',\n",
       "  'content': 'You are a coding assistant, you have the capability to assist with code-related tasks and modify files.'},\n",
       " {'role': 'user', 'content': 'Move trace_client function to bot.py.'},\n",
       " ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_9f2agXdE5nnUl1X04XkYHjeQ', function=Function(arguments='{\"description\":\"Move the \\'trace_client\\' function from its current location to the \\'bot.py\\' file.\"}', name='PlanCodeChange'), type='function')]),\n",
       " {'tool_call_id': 'call_9f2agXdE5nnUl1X04XkYHjeQ',\n",
       "  'role': 'tool',\n",
       "  'name': 'PlanCodeChange',\n",
       "  'content': \"I've read code and attempted to create code changes. The outcome of these code changes is as follows:\\n        - New files created: []\\n        - Existing files updated: [['Updated file autocoder/bot.py', 'Updated file autocoder/telemetry.py']]\\n        - Problems encountered: []\\n        ###\\n        Make the previous message more user-friendly\\n        \"}]"
      ]
     },
     "execution_count": 18,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "messages"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "2939b91a-fe15-4338-a52a-d4a7da7a1aca",
   "metadata": {},
   "outputs": [],
   "source": []
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "412d9f50-be8d-4f9c-b166-59ba5f876748",
   "metadata": {},
   "outputs": [],
   "source": []
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "dcf75302-039f-44d8-a7da-791176d6db85",
   "metadata": {},
   "outputs": [],
   "source": []
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3 (ipykernel)",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.9.18"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}

<END OF FILE>
####################
User Instruction:
Identify and remove any unused imports in the file autocoder/bot.py.'''

In [13]:
from autocoder.pydantic_models_v2 import create_implementation_plan
messages = [{"role": "user", "content": user_prompt}]

implementation_plan = create_implementation_plan.invoke(
    auto_coder.client,
    messages=messages,
    temperature=1,
    model=os.environ["MODEL"],
    stream=False,
    force=True,
)


In [15]:
implementation_plan[0].file_modifications

[FileModification(file_path='autocoder/bot.py', instruction='Remove unused import statements')]

In [14]:
implementation_plan

[ImplementationPlan(chain_of_thought='To identify and remove any unused imports in autocoder/bot.py, we need to first determine which imports are not being used in the file. This can be accomplished by analyzing the content of bot.py to see which imported modules and objects are not referenced anywhere in the code. Once we identify the unused imports, we can create a plan to remove these lines from the file.\n\nThe specific steps are to:\n1. Analyze the file autocoder/bot.py to find all the import statements.\n2. Determine which imports are not used in the rest of the file.\n3. Remove the lines containing the unused imports.\n4. Test to ensure that the removal of the imports does not affect the functionality of the script.\n', file_modifications=[FileModification(file_path='autocoder/bot.py', instruction='Remove unused import statements')], file_creations=[])]

In [17]:
from autocoder.pydantic_models_v2 import ImplementationPlan
print (ImplementationPlan.__doc__)

Represents a plan to implement a feature, a plan include file creation/modification operations.

    Example:
        plan = ImplementationPlan(
            chain_of_thought="Plan to refactor existing code.",
            file_modifications=[
                FileModification(
                    
                    blocks=[
        Block(
            step_by_step_instruction_rewrite_the_block="Refactor a method called `foo` that takes a parameter named `bar` and returns `bar`",
            start_newline_of_interested_block="def foo(bar):",
            end_newline_of_interested_block="return bar"
        )
        ]
                )
            ],
            file_creations=[
                FileCreation(
                    file_name="new_feature.py",
                    content="def new_function():
    # Add new feature code here",
                )
            ],
        )
    


In [18]:
implementation_plan[0].file_modifications[0].blocks[0].find_block(user_prompt)

'import os'

In [16]:
import difflib

# Sample strings for comparison
string1 = "apple pie"
string2 = "apples and pears"

# Calculate matching blocks
matcher = difflib.SequenceMatcher(None, string1, string2)
matching_blocks = matcher.get_matching_blocks()

# Extract matching substring
matching_substring = ""
for block in matching_blocks:
    i, j, length = block
    matching_substring += string1[i:i + length]

print(f"Matching Substring: {matching_substring}")


Matching Substring: apple pe


In [12]:
search_here[s[1].a: s[1].b + s[1].size]

' are awesome animal catterpillar who like other humans but not other caterpilar'

In [15]:
s.ratio()

0.21782178217821782